# Работа со строковыми значениями

Материалы:
* Макрушин С.В. Лекция 8: Работа со строковыми значениям
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
* https://tproger.ru/translations/regular-expression-python/
* https://realpython.com/nltk-nlp-python/

In [1]:
import re
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import csv

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lisa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lisa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [3]:
obj = {  
         'home_page': 'https://github.com/pypa/sampleproject',
         'keywords': 'sample setuptools development',
         'license': 'MIT'
      }
for k, v in obj.items():
      print(f'{k:10} = "{v}"')

home_page  = "https://github.com/pypa/sampleproject"
keywords   = "sample setuptools development"
license    = "MIT"


2. Дана строка 'aaa--bbb==ccc__ddd'. Написать регулярное выражение для разбивки строки на список ['aaa','bbb','ccc','ddd'].

In [4]:
pattern = re.compile(r'[a-z]{3}')
text = 'aaa--bbb==ccc__ddd'
pattern.findall(text)

['aaa', 'bbb', 'ccc', 'ddd']

3. Проверить корректность введенного E-mail

In [5]:
email = 'NVBlokhin@fa.ru'
pattern = re.compile(r'\w+@[a-z]+\.[a-z]{2,3}$')
pattern.match(email)

<re.Match object; span=(0, 15), match='NVBlokhin@fa.ru'>

4. Разбейте текст формулировки задачи 1 на слова.

In [6]:
text = """Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки."""
nltk.word_tokenize(text)[:5]

['Вывести', 'на', 'экран', 'данные', 'из']

In [7]:
nltk.sent_tokenize(text)

['Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках.',
 'Строковые литералы обернуть в кавычки.']

## Лабораторная работа 8

### Форматирование строк

1.1 Загрузите данные из файла `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в таблицу `recipes` При помощи форматирования строк выведите информацию об id рецепта и кол-ве игредиентов 5 случайных рецептов в виде таблицы следующего вида:

    
    |    id     | n_in  |
    |-------------------|
    |  400894   |  13   |
    |   68588   |   8   |
    |  362081   |   6   |
    |   53408   |  12   |
    |  221203   |   4   |

In [8]:
recipes = pd.read_csv('recipes_sample_with_tags_ingredients.csv', parse_dates=['submitted'], index_col=0)
rec = recipes.sample(5)
rec

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,tags,n_tags,indredients
12408,grandma s blueberry muffins,296315,35,396078,2008-04-03,11,this is one of my favorite recipes from my gra...,8,60-minutes-or-less;time-to-make;course;main-in...,14,milk*eggs*butter*flour*baking powder*vanilla*s...
8502,crispy french fries,219847,37,356062,2007-03-31,12,extra crispy lightly breaded french fries.,5,60-minutes-or-less;time-to-make;course;main-in...,28,cornstarch*milk*breadcrumbs*oil*potato
16343,london lemonade martini,429561,5,599450,2010-06-13,2,based on a recipe from ray foley’s the ultimat...,5,15-minutes-or-less;time-to-make;course;cuisine...,16,lemon juice*lime juice*gin*lemon twist*cointre...
11485,fried peppers onions and sausages,62511,40,80353,2003-05-19,8,i don't think the generic pork sausages found ...,7,60-minutes-or-less;time-to-make;course;main-in...,12,onions*olive oil*green bell peppers*garlic*red...
26635,sweet bacon wrapped hot dogs,470576,35,480970,2011-12-26,9,a coworker makes these for carry ins and they ...,3,60-minutes-or-less;time-to-make;course;main-in...,15,brown sugar*bacon*hot dogs


In [9]:
print(f"| {'id':^10} | {'n_in':^8} |")
print(f"|{'':-^23}|")
for i, line in rec.iterrows():
      print(f"| {line['id']:^10} | {line['n_ingredients']:^8} |")

|     id     |   n_in   |
|-----------------------|
|   296315   |    8     |
|   219847   |    5     |
|   429561   |    5     |
|   62511    |    7     |
|   470576   |    3     |


1.2 Напишите функцию `show_info`, которая для рецепта по его `id` создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название"

1. Шаг 1.
2. Шаг 2.
----------
#тэг1 #тэг2
```

    
Данные для создания строки получите из файлов `recipes_sample_with_tags_ingredients.csv`, `steps_sample.xml` (__ЛР4__) и `tags_sample.csv` (__ЛР5__). 
Выведите созданную строку на экран.

In [10]:
with open('steps_sample.xml', 'r', encoding='utf8') as fp:
    data = BeautifulSoup(fp, 'xml')

In [11]:
def show_info(idd):
    s = f"{recipes[recipes['id'] == idd]['name'].values[0].capitalize()}\n"
    for rec in data.find_all('recipe'):  
        if rec.id.text == str(idd):
            max_len = 0
            for i, step in enumerate(rec.find_all('step'), start=1):
                s += f'{i}. {step.text.capitalize()}.\n'
                if len(f'{i}. {step.text.capitalize()}.') > max_len:
                    max_len = len(f'{i}. {step.text.capitalize()}.')
    s += f"{'':-^{max_len}}\n"
    with open('tags_sample.csv') as fp:
        reader = csv.reader(fp)
        next(reader)
        for row in reader:
            row[0] = int(row[0])
            if row[0] == idd:
                s += f'#{row[1]} '
    return s
st_rec = show_info(136539)
print(st_rec)

Chicken malai kebab
1. If you are using wooden or bamboo skewers , be sure to soak them in water so that they dont splinter or burn on you.
2. Mix together the ginger , garlic , salt and pepper.
3. Rub this mix into the chicken and set aside for at least 15 minute.
4. Mix together the egg , cheese , chillies , coriander , cream , mace , nutmeg and cornstarch.
5. Rub this mixture into the chicken as well.
6. Marinade for a minimum of 3 hours.
7. Thread the chicken strips onto the skewers.
8. If you are putting more than one piece on a skewer , leave them spaced about 1 in apart.
9. Grill the kebabs for 5 min , or roast in the oven @ 275 degrees , or place in the rotisserie according to appliance directions.
10. If you are using the rotisserie , you may roast them until they are done.
11. If you are grilling or oven roasting them , remove them from the heat and place them on a rack to allow the excess liquid to drip off.
12. Then roast again for 3 minutes.
-------------------------------

## Работа с регулярными выражениями

В задачах данного блока подразумевается, что вы не будете использовать никаких строковые методы (`split`, `startswith` и т.д.). Все задачи необходимо решить при помощи регулярных выражений.

2.1 Посчитайте кол-во отзывов, в которых встречаются числа.

In [12]:
reviews = pd.read_csv('reviews_sample.csv', index_col=0)
reviews.reset_index(drop = True, inplace=True)
reviews.head()

,user_id,recipe_id,date,rating,review
0,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [13]:
pattern = re.compile(r'\d+')
k = 0
for text in reviews['review'].values:
    if not pd.isna(text) and pattern.findall(text):
        k += 1
print(k)

49246


2.2 Найдите все смайлики в отзывах. Смайлик состоит из трех частей: глаза (символ `=` или `:`), нос (символ `-`), губы (символ `)` или `(`). Смайлик может иметь вид "глаза-нос-губы" или "губы-нос-глаза". Нос может отсутствовать.

In [14]:
pattern = re.compile(r'((?:[=:]\-?[\(\)])|(?:[\(\)]\-?[=:]))')
for text in reviews['review'].values:
    if not pd.isna(text) and pattern.findall(text):
        print(pattern.findall(text))

[':)']
[':)', ':)']
[':)']
[':-)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':-)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':-)']
[':)']
[':)', ':)']
[':)']
[':)', ':)', ':)']
[':)']
['=)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)', ':)']
[':)']
[':)']
[':)']
[':)']
[':)', ':)']
[':-)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)']
['):']
[':)']
['=)']
[':)']
[':-)']
[':(']
[':)']
[':)']
[':(']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)', ':)']
[':)']
[':)']
[':-)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':-)']
[':)']
[':)']
[':)']
[':)']
[':)']
[':)', ':)', ':)']
[':-)']
[':)']
[':)']
[':)']
[':)']
['=)']
[':-)']
[':)']
[':)']
[':)']
[':)']
[':-)', ':-)']
[':)']
[':)']
[':)']
[':)']
['=)']
[':)']
[':)']
[':)']
['=)']
[':)']
[':)']
['=)']
[':)']
[':)']
['=)']
[':)']
['):']
['=)']
[':-)']
[':)']
[':)']
[':)']
['=)']
[':)', ':)']
[':)']
[':)']
[':)']
[':(']
[':)']
[':)']
[':)']
[':)']
[':(']
[':)']
[':)']
[':(']
[':)']
[':)']
[':)'

2.3 Проверьте, что все даты в датасете имеют вид "YYYY-MM-DD". Продемонстрируйте работу вашего решения, приведя пример из датасета и контрпример не из датасета.

In [15]:
reviews['date'].values

array(['2003-05-01', '2007-09-16', '2008-01-10', ..., '2008-04-08',
       '2015-06-02', '2010-09-30'], dtype=object)

In [16]:
pattern = re.compile(r'([12]\d{3}\-(?:(?:0[1-9])|(?:1[0-2]))\-(?:(?:0[1-9])|(?:[1-2][0-9])|(?:[3][01])))')
for text in reviews['date'].values:
    if not pattern.findall(text):
        print(text)
pattern.findall('09.12.2008')

[]

2.4 Используя строку-результат задачи 1.2, найдите первое слово каждого шага в рецепте

In [17]:
pattern = re.compile(r'[1-9]\d*\. ([A-Z][a-z]+)')
pattern.findall(st_rec)

['If',
 'Mix',
 'Rub',
 'Mix',
 'Rub',
 'Marinade',
 'Thread',
 'If',
 'Grill',
 'If',
 'If',
 'Then']

2.5 Используя регулярные выражения, удалите из описаний все символы, кроме английских букв, цифр и пробелов. Сохраните предобработанные описания в файл `preprocessed_descriptions.csv`, содержащий 2 столбца: `name` и `preprocessed_descriptions`.

In [18]:
preprocessed_descriptions = recipes[['name', 'description']]
preprocessed_descriptions.head()

,name,description
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,"these were so go, it surprised even me."
3,italian gut busters,my sister-in-law made these for us at a family...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...


In [19]:
preprocessed_descriptions = recipes[['name', 'description']].copy()
pattern = re.compile(r'[^A-Za-z0-9 ]')
for i, text in enumerate(preprocessed_descriptions['description'].values):
    if not pd.isna(text):
        preprocessed_descriptions['description'][i] = pattern.sub("", text)

In [20]:
preprocessed_descriptions.rename(columns={'description': 'preprocessed_descriptions'}, inplace=True)
preprocessed_descriptions.head()

,name,preprocessed_descriptions
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,these were so go it surprised even me
3,italian gut busters,my sisterinlaw made these for us at a family g...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...


In [21]:
preprocessed_descriptions.to_csv('preprocessed_descriptions.csv', encoding='utf-8', index=False)

### Сегментация текста

3.1 Разбейте отзывы из файла `recipes_sample.csv` (__ЛР2__) на предложения, а предложения - на слова (используйте `sent_tokenize` и `word_tokenize` из `nltk`). Каждый отзыв представьте в виде списка списков: внешний список - предложения, вложенные списки - слова в предложении. Исключите знаки препинания из списков слов.

`'Предложение номер один. Предложение номер два.' => [['Предложение', 'номер', 'один'], ['Предложение', 'номер', 'два']]`

In [22]:
reviews = pd.read_csv('reviews_sample.csv', index_col=0)
reviews.reset_index(drop = True, inplace=True)
reviews.head()

,user_id,recipe_id,date,rating,review
0,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [23]:
pattern = re.compile(r'[\W]+$')
reviews['tokenized'] = reviews.apply(lambda row: [[word for word in nltk.word_tokenize(sent) if not pattern.match(word)] for sent in nltk.sent_tokenize(row['review'])] if not pd.isna(row['review']) else [], axis=1)

In [24]:
reviews.head()

,user_id,recipe_id,date,rating,review,tokenized
0,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,"[[Last, week, whole, sides, of, frozen, salmon..."
1,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...,"[[So, simple, and, so, tasty], [I, used, a, ye..."
2,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy...","[[Very, nice, breakfast, HH, easy, to, make, a..."
3,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...,"[[These, are, a, favorite, for, the, holidays,..."
4,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...,"[[Excellent, soup], [The, tomato, flavor, is, ..."


3.2 Посчитайте кол-во уникальных слов среди всех отзывов (без учета регистра и знаков препинания).

In [25]:
uniq = set()
for text in reviews['tokenized']:
    for line in text:
        uniq.update([word.lower() for word in line])
len(uniq)

66790

3.3 Найдите 5 самых длинных (по количеству слов; без учета знаков препинания) отзывов в датасете и выведите их в порядке убывания длины.

In [26]:
reviews['tok_len'] = reviews.apply(lambda row: sum([len(line) for line in row['tokenized']]), axis=1) 
reviews.head()

,user_id,recipe_id,date,rating,review,tokenized,tok_len
0,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,"[[Last, week, whole, sides, of, frozen, salmon...",104
1,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...,"[[So, simple, and, so, tasty], [I, used, a, ye...",50
2,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy...","[[Very, nice, breakfast, HH, easy, to, make, a...",45
3,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...,"[[These, are, a, favorite, for, the, holidays,...",24
4,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...,"[[Excellent, soup], [The, tomato, flavor, is, ...",45


In [27]:
reviews.sort_values(by="tok_len", ascending=False)[:5]['review']

41791     One of my sisters was in need of some comfort ...
92161     I am in the process of prepping all of my ingr...
87799     I made these for a Valentine's day dinner and ...
126122    I made these, and was worried that they would ...
19051     Your vet has severly mislead you and this diet...
Name: review, dtype: object

3.4 Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на любом предложении из отзывов.


In [28]:
def part_of_speach(text):
    text = nltk.word_tokenize(text)
    for word, part in nltk.pos_tag(text):
        print(f"{part:^{len(word)|len(part)}} ", end="")
    print("")
    for word, part in nltk.pos_tag(text):
        print(f"{word:^{len(word)|len(part)}} ", end="")

In [29]:
part_of_speach('I omitted the raspberries and added strawberries instead')

PRP   VBD   DT      NNS     CC    VBD         NNS         RB    
 I  omitted the raspberries and  added   strawberries   instead 

In [30]:
part_of_speach(nltk.sent_tokenize(reviews['review'][0])[1])

  DT     NN   VBZ   JJ    IN    JJ     NN   ,   RB     IN   PRP   VBZ   IN    NN     NNS   . 
 This  recipe is  perfect for salmon fillet ,  even  though it   calls  for salmon steaks  . 